In [1]:
import numpy as np
n_features = 'ascvd'
term_pred = 'short_term'
data_file_name = 'data_y10_ascvd'
training_id_file_name = 'all_training_ID_outerloop_cohort_0_10'
endpt = 10;
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
load_dir


'/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2/csv_files'

In [7]:
data_full

,ID,event,time,ascvd,sex,race,age,cholesterol,hdl,sbp,hbp.medication,smoker,diabetes
0,100016012504,0,3689.025,0.015689,0,0,27,186.0,38.0,134.000000,0,1,0
1,100023004268,0,3689.025,0.003673,0,0,35,166.0,43.0,112.000000,0,0,0
2,100033323702,0,3689.025,0.001691,0,0,28,234.0,52.0,100.000000,0,0,0
3,100048415270,0,1169.000,0.031799,0,1,35,137.0,40.0,118.000000,0,1,0
4,100056526386,0,3689.025,0.022868,0,1,34,199.0,44.0,132.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,0.004976,1,0,35,225.0,57.0,120.000000,0,0,0
4310,416771521620,0,3689.025,0.008172,0,1,28,204.0,76.0,106.000000,0,0,0
4311,416783315386,0,3689.025,0.001775,1,0,32,159.0,58.0,120.000000,0,0,0
4312,416796224310,0,3689.025,0.000524,1,1,32,164.0,55.0,110.843678,0,0,0


In [8]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [9]:
data_full

,ID,event,time,0,1,2,3,4,5,6,7,8,9
0,100016012504,0,3689.025,0.311316,-1.106215,-0.970306,-0.818280,0.235193,-1.090061,1.834180,-0.126551,1.580754,-0.150863
1,100023004268,0,3689.025,-0.176267,-1.106215,-0.970306,1.381820,-0.356134,-0.732871,0.255030,-0.126551,-0.632609,-0.150863
2,100033323702,0,3689.025,-0.256705,-1.106215,-0.970306,-0.543268,1.654379,-0.089929,-0.606324,-0.126551,-0.632609,-0.150863
3,100048415270,0,1169.000,0.965055,-1.106215,1.030602,1.381820,-1.213559,-0.947185,0.685708,-0.126551,1.580754,-0.150863
4,100056526386,0,3689.025,0.602639,-1.106215,1.030602,1.106808,0.619556,-0.661433,1.690621,-0.126551,-0.632609,-0.150863
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,-0.123415,0.903984,-0.970306,1.381820,1.388282,0.267261,0.829267,-0.126551,-0.632609,-0.150863
4310,416771521620,0,3689.025,0.006276,-1.106215,1.030602,-0.543268,0.767388,1.624583,-0.175647,-0.126551,-0.632609,-0.150863
4311,416783315386,0,3689.025,-0.253303,0.903984,-0.970306,0.556783,-0.563099,0.338699,0.829267,-0.126551,-0.632609,-0.150863
4312,416796224310,0,3689.025,-0.304080,0.903984,1.030602,0.556783,-0.415267,0.124385,0.172030,-0.126551,-0.632609,-0.150863


In [ ]:
# Outerloop:
for fold in range(1,25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir,'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 1:

Using nesterov accelerated gradient
training model
cost: 0.819561, iteration: 0
cost: 0.814730, iteration: 23
cost: 0.808156, iteration: 46
cost: 0.801484, iteration: 69
cost: 0.794859, iteration: 92
cost: 0.788294, iteration: 115
cost: 0.781788, iteration: 138
cost: 0.775343, iteration: 161
cost: 0.768956, iteration: 184
cost: 0.762629, iteration: 207
cost: 0.756359, iteration: 230
cost: 0.750146, iteration: 253
cost: 0.743991, iteration: 276
cost: 0.737892, iteration: 299
cost: 0.731849, iteration: 322
cost: 0.725862, iteration: 345
cost: 0.719930, iteration: 368
cost: 0.714051, iteration: 391
cost: 0.708227, iteration: 414
cost: 0.702457, iteration: 437
cost: 0.696739, iteration: 460
cost: 0.691073, iteration: 483
cost: 0.685460, iteration: 506
cost: 0.679898, iteration: 529
cost: 0.674387, iteration: 552
cost: 0.668927, iteration: 575
cost: 0.663517, iteration: 598
cost: 0.658156, iteration: 621
cost: 0.652845, iteration: 644
cost: 0.647582, iteration: 667
cost: 0.642368,